# 使用 4o-mini 模拟 OpenAI One 的推理过程


我们的目标是创建一个 AI 助手，能够像 OpenAI One 一样，**逐步展示推理过程**，最终给出答案。**4o-mini** 是一个轻量级的大型语言模型。虽然它的性能不及 OpenAI One，但通过巧妙的提示设计和代码实现，我们可以让它模拟类似的推理过程。

### COT设计思路

- **逐步推理**：助手将问题分解为多个步骤，每个步骤都有标题和详细内容。
- **动态决策**：在每一步，助手决定是继续推理还是给出最终答案。
- **JSON 格式**：为了方便解析和展示，助手的每个响应都采用 JSON 格式。

---

### 局限性

1、o1并不是简单通过提示工程就升级出来的，而是加入了后训练推理阶段：

- 例如MCTS等算法被用作一个工具来构建细粒度的奖励信号
- 这些奖励信号可以指导模型的后训练训练推理过程

![image](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/28777116_249bd793-3578-4a05-874a-bf391f1b3abd.png)

MCTS(Monte Carlo Tree Search, 蒙特卡洛树搜索)，扩展之前的算法不只是传统意义上的不是传统的决策树搜索工具，而是用来引导和生成合理的推理过程树搜索的层次结构被用来模拟推理的层次结构，从而帮助模型学习如何进行更有条理的思考。这些推理过程可以被转化为token序列，进而产生细粒度（token级，句子级）的奖励信号。



2、除此之外，CoT 虽然也能生成中间推理步骤，但没有教会模型"如何从内部深入思考问题的关联"

STAR (STaR) 是一个利用语言模型进行推理的框架，通过 Bootstrap 方式生成推理过程(Rationales)，将推理过程融入到训练过程中，让模型学会进行推理

1. 核心思路：
- 利用大语言模型(LLM)已有的推理能力
- 通过 Bootstrap 方式生成推理过程(Rationales)
- 将推理过程融入到训练过程中，让模型学会进行推理

2. 工作流程：
- 输入问题(Question)
- 语言模型生成推理过程(Rationale)和答案(Answer)
- 系统会验证答案的正确性
- 如果答案错误，会提供提示(Hint)进行修正

这个框架的名称来源于论文 "STaR: Bootstrapping Reasoning With Reasoning"，主要创新点在于通过自举(Bootstrapping)方式来增强模型的推理能力。

当然，以上都是猜测，o1并没有官方解释说明
### 环境准备

首先，确保安装了必要的库和模型。
- 知识准备 希望大家先完成4o的OPENAI API基本调用的学习，因为这是个小实战项目

In [1]:
!pip install openai pydantic


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 导入必要的库

In [2]:
from enum import Enum
import openai
import json
import time
from pydantic import BaseModel

### 设置 OpenAI API

In [3]:
# 设置 OpenAI API 密钥（请替换为您的实际密钥）
# client = openai.OpenAI(api_key="YOUR_API_KEY")
client = openai.OpenAI()

### 定义JSON格式及大模型的行为

我们需要定义助手的行动类型和响应为JSON格式从而方便在上下游传输数据流。使用4o大模型的定义结构的response_format参数。

`response_format` 告诉 OpenAI 模型以特定的结构输出它的回复，就像填写一个表格一样。它避免了模型随意发挥，确保你得到的数据格式正确，可以直接使用，即我们常说的Json结构。

**简单来说，`response_format`  就像一个模具，让模型的回复符合你的要求。**

**`response_format` 怎么用？**

1. **定义结构：** 你需要先定义你想要的输出结构，就像设计一个表格，包括字段名称和数据类型（例如：姓名-字符串，年龄-数字，日期-日期）。 你可以用 JSON Schema，或者 Python 的 Pydantic 和 JavaScript 的 Zod 库来定义。

2. **API 调用：**  在调用 OpenAI API 时，把定义好的结构通过 `response_format` 参数传递给模型。

3. **使用数据：** 因为模型的回复已经符合你定义的结构，你可以直接在程序中安全地使用这些数据。

具体在 https://platform.openai.com/docs/guides/structured-outputs/supported-schemas


In [4]:
# 定义助手的行动类型
class ActionType(str, Enum):
    CONTINUE = 'continue'
    FINAL_ANSWER = 'final_answer'

# 定义助手的响应格式
class Answer(BaseModel):
    title: str
    next_action: ActionType
    content: str

### 定义与模型交互的函数

这个函数用于与 4o-mini 模型进行交互，发送请求并处理响应。

In [13]:
def make_api_call(messages, max_tokens, is_final_answer=False):
    """
    发送请求到 4o-mini 模型，并处理可能的异常。
    """
    for attempt in range(3):
        try:
            response = client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=max_tokens,
                response_format=Answer
            )
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {
                        "title": "错误",
                        "content": f"尝试3次后无法生成最终答案。错误信息: {str(e)}"
                    }
                else:
                    return {
                        "title": "错误",
                        "content": f"尝试3次后无法生成步骤。错误信息: {str(e)}",
                        "next_action": "final_answer"
                    }
            time.sleep(1)  # 等待 1 秒后重试

### 构建逐步生成推理步骤的函数

这个函数是核心，根据用户提供的 prompt，通过迭代调用API，逐步推理并最终生成答案。它最多进行五个推理步骤，每个步骤都会生成一个包含标题、内容和下一步操作（继续或最终答案）的JSON格式数据。

* **逐步推理:**  函数的核心是逐步推理，每个步骤都建立在前一步的基础上。
* **JSON格式:**  API调用返回的数据和助手回复都是JSON格式，包含 `title`、`content` 和 `next_action`。
* **最多五步:**  推理步骤最多进行五次。
* **最终答案请求:**  在推理步骤完成后，显式请求助手给出最终答案。
* **时间记录:**  记录每个步骤和最终答案的耗时。

In [14]:
def generate_response(prompt):
    """
    根据用户提示，逐步生成推理步骤并最终给出答案。
    """
    # 初始化消息列表
    messages = [
        {
            "role": "system",
            "content": (
                "你是一个专家级的AI助手，能够逐步解释你的推理过程。"
                "对于每一步，提供一个描述该步骤的标题和详细内容。"
                "决定是否需要继续下一步，或是准备给出最终答案。"
                "请以JSON格式响应，包含 'title'、'content' 和 'next_action'（'continue' 或 'final_answer'）。"
                "请使用至少3个推理步骤。"
            )
        },
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "好的，我将逐步分析并回答您的问题。"}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while step_count <= 5:
        start_time = time.time()
        # 调用API获取当前步骤的数据
        step_data = make_api_call(messages, max_tokens=500)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        # 添加当前步骤的信息
        steps.append(
            {
                "步骤": step_count,
                "标题": step_data.get('title', '无标题'),
                "内容": step_data.get('content', ''),
                "耗时": f"{thinking_time:.2f} 秒"
            }
        )

        # 更新消息列表，供下一次迭代使用
        messages.append(
            {
                "role": "assistant",
                "content": json.dumps(step_data, ensure_ascii=False)
            }
        )

        # 实时显示当前步骤
        print(f"\n{'=' * 40}\n步骤 {step_count}: {step_data.get('title', '无标题')}\n{'-' * 40}")
        print(step_data.get('content', ''))
        print(f"此步骤耗时: {thinking_time:.2f} 秒")

        # 判断是否需要继续或结束
        if step_data.get('next_action') == 'final_answer':
            break

        step_count += 1

    # 请求生成最终答案
    messages.append({"role": "user", "content": "请根据以上推理给出最终答案。"})

    start_time = time.time()
    final_data = make_api_call(messages, max_tokens=200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    # 添加最终答案的信息
    steps.append(
        {
            "步骤": "最终答案",
            "内容": final_data.get('content', ''),
            "耗时": f"{thinking_time:.2f} 秒"
        }
    )

    # 显示最终答案
    print(f"\n{'=' * 40}\n最终答案\n{'-' * 40}")
    print(final_data.get('content', ''))
    print(f"最终答案耗时: {thinking_time:.2f} 秒")

    # 返回所有步骤和总思考时间
    return steps, total_thinking_time



定义一个主函数来组织逻辑，运行整个程序。

In [15]:
def main():
    """
    主函数，负责获取用户输入并生成响应。
    """
    print("==========================================")
    print(" 使用 4o-mini 模拟 OpenAI One 的推理过程")
    print("==========================================\n")

    print("这是一个使用 4o-mini 创建类似 OpenAI One 推理链的原型。\n")

    # 获取用户查询
    user_query = input("请输入您的查询（例如：单词 'strawberry' 中有多少个 'R'？）:\n> ").strip()

    if user_query:
        print("\n正在生成响应...\n")

        # 生成并显示响应
        steps, total_thinking_time = generate_response(user_query)


### 运行程序

In [16]:
main()

 使用 4o-mini 模拟 OpenAI One 的推理过程

这是一个使用 4o-mini 创建类似 OpenAI One 推理链的原型。


正在生成响应...


步骤 1: 步骤 1: 理解二极管发光的原理
----------------------------------------
二极管是一种电子元件，特别是LED（二极管发光二极管），其工作原理是当电流通过时会发光。需要确保二极管的极性连接正确，即阳极接正极，阴极接负极。
此步骤耗时: 1.71 秒

步骤 2: 步骤 2: 找到合适的电源
----------------------------------------
为了让二极管发亮，我们需要一个可提供电流的简单电源。常见的选择包括：电池（如AA电池）、USB接口、电源适配器等。确保电源的电压与二极管的额定前向电压匹配。对于LED，通常在2V到3V之间。
此步骤耗时: 1.24 秒

步骤 3: 步骤 3: 连接二极管与电源
----------------------------------------
将二极管的阳极（长脚）连接到电源的正极，阴极（短脚）连接到电源的负极。如果使用电池，直接将二极管与电池连接即可。确保电流流动方向正确，这样二极管就会发光。
此步骤耗时: 1.23 秒

最终答案
----------------------------------------
你可以将二极管（如LED）连接到电池的正极和负极之间来让它发光。具体步骤是：1) 确保二极管的阳极（长脚）连接到电池的正极，阴极（短脚）连接到负极。2) 选择合适的电源（比如AA电池），确保电源电压与二极管工作电压相匹配。这样连接后，二极管应该会发光。
最终答案耗时: 1.45 秒


In [17]:
response = client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": "可以把二极管插在什么地方能让他发亮，不要专业的设备，比如苹果？"}],
            )
print(response.choices[0].message.content)

要让二极管发亮，只需要供给它适当的电压和电流。你可以尝试以下几种简单的方法，而不需要专业的设备：

1. **电池**：使用一个干电池（如AA或9V电池）和适当的电阻器，可以将二极管连接到电池的正负极。确保电路中有合适的电阻，以防止二极管过流烧毁。

2. **LED手电筒**：许多LED手电筒中的电路可以直接连接LED二极管，如果能拆解手电筒并接入二极管，可能可以使二极管发亮。

3. **电源适配器**：一些小型的电源适配器可以用来为二极管供电。确保电压符合二极管的额定值。

4. **USB接口**：如果你有USB接口的供电设备（例如手机充电器），可以借助USB转接线，将二极管通过电阻与USB接口连接，利用5V电源使二极管发亮。

5. **旧电子设备**：可以尝试从一些旧的电子设备（如玩具、旧手机等）中取出供电源，连接到二极管。

确保在操作过程中遵循安全指引，避免短路和过热。


--- ++

通过以上步骤，我们成功地使用 **4o-mini** 模型模拟了 **OpenAI One** 的逐步推理过程。尽管模型性能有所差距，但通过巧妙的提示设计和代码实现，我们能够让模型展示出类似的推理能力。